# Unemployment rate from the [BLS API](https://www.bls.gov/developers/)

#### Load Python tools and environment variables

In [1]:
import os
import json
import requests
import pandas as pd
import jupyter_black

In [2]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [3]:
today = pd.Timestamp("today").strftime("%Y_%m_%d")

In [4]:
api_key = os.environ.get("BLS_API_KEY")
api_key

'48d12e7fd9cb4ca5a2cff674b269dc71'

---

## Historical data

#### The API will only deliver limited years, so [import historical](https://data.bls.gov/timeseries/CUUR0000SA0), too

In [75]:
historical_src = (
    pd.read_csv(
        "data/processed/unemployment_rate_latest.csv",
        parse_dates=["date"],
        dtype={"year": str},
    )
    .sort_values("date")
    .reset_index(drop=True)
)

In [76]:
historical_src.head()

,date,year,value,month,month_year,series
0,1948-01-01,1948,3.4,January,1948-01,LNS14000000
1,1948-02-01,1948,3.8,February,1948-02,LNS14000000
2,1948-03-01,1948,4.0,March,1948-03,LNS14000000
3,1948-04-01,1948,3.9,April,1948-04,LNS14000000
4,1948-05-01,1948,3.5,May,1948-05,LNS14000000


---

## Current data

In [7]:
variable = "LNS14000000"

#### Start and end dates for request

In [8]:
start = "2023"
end = "2024"

#### Request params

In [9]:
headers = {"Content-type": "application/json"}

data = json.dumps(
    {
        # This is the CPI variable
        "seriesid": [variable],
        "startyear": start,
        "endyear": end,
        "registrationkey": api_key,
    }
)

#### Get data from API

In [10]:
p = requests.post(
    "https://api.bls.gov/publicAPI/v2/timeseries/data/", data=data, headers=headers
)
json_data = json.loads(p.text)

In [11]:
data

'{"seriesid": ["LNS14000000"], "startyear": "2023", "endyear": "2024", "registrationkey": "48d12e7fd9cb4ca5a2cff674b269dc71"}'

#### Into a dataframe

In [132]:
src = pd.DataFrame(json_data["Results"]["series"][0]["data"])[
    ["year", "period", "periodName", "value"]
].rename(columns={"periodName": "month"})

In [134]:
src["series"] = "LNS14000000"
src["value"] = src["value"].astype(float)
src["date"] = pd.to_datetime(
    src["year"] + "-" + src["period"].str.replace("M", "") + "-01-"
)
src["month_year"] = src["year"] + "-" + src["period"].str.replace("M", "")
src = src.drop("period", axis=1)

---

## Combine

#### Concatenate the historical data with the most recent data

In [142]:
df = (
    pd.concat([src, historical_src])
    .drop_duplicates()
    .sort_values("date")
    .reset_index(drop=True)
)

In [145]:
df["value"] = df["value"].astype(float)

---

## Toplines

#### Latest value

In [149]:
latest_value = df[df["date"] == df["date"].max()]["value"].astype(float).iloc[0]
latest_value

3.7

#### Previous month

In [150]:
previous_value = df["value"].astype(float).iloc[-2]
previous_value

3.7

#### Latest month/year

In [151]:
latest_month = df[df["date"] == df["date"].max()]["month"].iloc[0]
latest_year = df[df["date"] == df["date"].max()]["year"].iloc[0]
latest_period = f"{latest_month} {latest_year}"
latest_period

'January 2024'

#### Change? 

In [152]:
change = (latest_value - previous_value).round(1)
change

0.0

In [157]:
if change == 0:
    change_description = "remaining the same as the"
elif change > 0:
    change_description = f"increasing by {change} percentage points over the"
else:
    change_description = f"decreasing by {change} percentage points over the"

In [158]:
change_description

'remaining the same as the'

In [159]:
f"The US unemployment rate in {latest_period} was {latest_value}%, {change_description} previous month's rate of {previous_value}%."

"The US unemployment rate in January 2024 was 3.7%, remaining the same as the previous month's rate of 3.7%."

---

## Exports

In [26]:
df["month_year"] = df["month_year"].astype(str)
df["date"] = df["date"].astype(str)

#### CSV

In [27]:
df.to_csv(f"data/processed/unemployment_rate_latest.csv", index=False)
df.to_csv(f"data/processed/unemployment_rate_latest_{today}.csv", index=False)

#### JSON

In [28]:
df.to_json(f"data/processed/unemployment_rate_latest.json", indent=4, orient="records")
df.to_json(
    f"data/processed/unemployment_rate_latest_{today}.json", indent=4, orient="records"
)